<a href="https://colab.research.google.com/github/vikasharma005/40-ML-Projects/blob/main/Text_Generation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from __future__ import print_function
from tensorflow.keras.callbacks import LambdaCallback
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation
from tensorflow.keras.layers import LSTM
from tensorflow.keras.optimizers import RMSprop
import numpy as np
import random
import sys

In [ ]:
!wget -O sherlock_homes.txt http://www.gutenberg.org/files/1661/1661-0.txt

--2024-09-25 05:29:52--  http://www.gutenberg.org/files/1661/1661-0.txt
Resolving www.gutenberg.org (www.gutenberg.org)... 152.19.134.47, 2610:28:3090:3000:0:bad:cafe:47
Connecting to www.gutenberg.org (www.gutenberg.org)|152.19.134.47|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://www.gutenberg.org/files/1661/1661-0.txt [following]
--2024-09-25 05:29:52--  https://www.gutenberg.org/files/1661/1661-0.txt
Connecting to www.gutenberg.org (www.gutenberg.org)|152.19.134.47|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 607504 (593K) [text/plain]
Saving to: ‘sherlock_homes.txt’

sherlock_homes.txt  100%[===================>] 593.27K  --.-KB/s    in 0.06s   

2024-09-25 05:29:52 (9.48 MB/s) - ‘sherlock_homes.txt’ saved [607504/607504]



In [ ]:
text = open('sherlock_homes.txt', 'r').read().lower()
print('text length', len(text))

text length 581425


In [ ]:
print(text[:1000])

﻿the project gutenberg ebook of the adventures of sherlock holmes,
by arthur conan doyle

this ebook is for the use of anyone anywhere in the united states and
most other parts of the world at no cost and with almost no restrictions
whatsoever. you may copy it, give it away or re-use it under the terms
of the project gutenberg license included with this ebook or online at
www.gutenberg.org. if you are not located in the united states, you
will have to check the laws of the country where you are located before
using this ebook.

title: the adventures of sherlock holmes

author: arthur conan doyle

release date: november 29, 2002 [ebook #1661]
[most recently updated: october 10, 2023]

language: english

character set encoding: utf-8

produced by: an anonymous project gutenberg volunteer and jose menendez

*** start of the project gutenberg ebook the adventures of sherlock
holmes ***




the adventures of sherlock holmes

by arthur conan doyle


contents

   i.     a scandal in bohemia
 

# Map chars to integers

In [ ]:
chars = sorted(list(set(text)))
print('total chars: ', len(chars))

total chars:  72


In [ ]:
char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))

# Split up into subsequences

In [ ]:
maxlen = 40
step = 3
sentences = []
next_chars = []
for i in range(0, len(text) - maxlen, step):
    sentences.append(text[i: i + maxlen])
    next_chars.append(text[i + maxlen])
print('nb sequences:', len(sentences))

nb sequences: 193795


In [ ]:
print(sentences[:3])
print(next_chars[:3])

['\ufeffthe project gutenberg ebook of the adve', 'e project gutenberg ebook of the adventu', 'roject gutenberg ebook of the adventures']
['n', 'r', ' ']


In [ ]:
x = np.zeros((len(sentences), maxlen, len(chars)), dtype=bool)
y = np.zeros((len(sentences), len(chars)), dtype=bool)
for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        x[i, t, char_indices[char]] = 1
    y[i, char_indices[next_chars[i]]] = 1

In [ ]:
print(x[:3])
print(y[:3])

[[[False False False ... False False  True]
  [False False False ... False False False]
  [False False False ... False False False]
  ...
  [False False False ... False False False]
  [False False False ... False False False]
  [False False False ... False False False]]

 [[False False False ... False False False]
  [False  True False ... False False False]
  [False False False ... False False False]
  ...
  [False False False ... False False False]
  [False False False ... False False False]
  [False False False ... False False False]]

 [[False False False ... False False False]
  [False False False ... False False False]
  [False False False ... False False False]
  ...
  [False False False ... False False False]
  [False False False ... False False False]
  [False False False ... False False False]]]
[[False False False False False False False False False False False False
  False False False False False False False False False False False False
  False False False False False Fals

# Building Model
In this model a small recurrent neural networks is used for both simplicity and because of the training time but if we want to train a more sophisticated model we can increase the size of the network. We can also use a model pretrained on some other text like wikipedia text to both speed up the training process and get better results.

In [ ]:
model = Sequential()
model.add(LSTM(128, input_shape=(maxlen, len(chars))))
model.add(Dense(len(chars)))
model.add(Activation('softmax'))

/usr/local/lib/python3.10/dist-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [ ]:
optimizer = RMSprop(learning_rate=0.01)
model.compile(loss='categorical_crossentropy', optimizer=optimizer)

In [ ]:
def sample(preds, temperature=1.0):
    # helper function to sample an index from a probability array
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

In [ ]:
def on_epoch_end(epoch, logs):
    # Function invoked at end of each epoch. Prints generated text.
    print()
    print('----- Generating text after Epoch: %d' % epoch)

    start_index = random.randint(0, len(text) - maxlen - 1)
    for diversity in [0.2, 0.5, 1.0, 1.2]:
        print('----- diversity:', diversity)

        generated = ''
        sentence = text[start_index: start_index + maxlen]
        generated += sentence
        print('----- Generating with seed: "' + sentence + '"')
        sys.stdout.write(generated)

        for i in range(400):
            x_pred = np.zeros((1, maxlen, len(chars)))
            for t, char in enumerate(sentence):
                x_pred[0, t, char_indices[char]] = 1.

            preds = model.predict(x_pred, verbose=0)[0]
            next_index = sample(preds, diversity)
            next_char = indices_char[next_index]

            generated += next_char
            sentence = sentence[1:] + next_char

            sys.stdout.write(next_char)
            sys.stdout.flush()
        print()
print_callback = LambdaCallback(on_epoch_end=on_epoch_end)

# Defining callbacks

In [ ]:
from tensorflow.keras.callbacks import ModelCheckpoint

filepath = "weights.keras"
checkpoint = ModelCheckpoint(filepath, monitor='loss',
                             verbose=1, save_best_only=True,
                             mode='min')

In [ ]:
from tensorflow.keras.callbacks import ReduceLROnPlateau
reduce_lr = ReduceLROnPlateau(monitor='loss', factor=0.2,
                              patience=1, min_lr=0.001)

In [ ]:
callbacks = [print_callback, checkpoint, reduce_lr]

# Training the model

In [ ]:
model.fit(x, y, batch_size=128, epochs=5, callbacks=callbacks)

Epoch 1/5
1514/1515 ━━━━━━━━━━━━━━━━━━━━ 0s 103ms/step - loss: 2.3095
----- Generating text after Epoch: 0
----- diversity: 0.2
----- Generating with seed: "“it is a noiseless lock,” said he. “it i"
“it is a noiseless lock,” said he. “it is the crost of the crararal the cranger have the crose the croor and the trount and the crararary and the crones and he had been the cranger and he had sheet and he have the rark which had heard the cronning and the crost and which street and the crance the crarust and the crost and the crark and in the crount and the stranger the crones and the crark and the cranger which it the crarking the cros
----- diversity: 0.5
----- Generating with seed: "“it is a noiseless lock,” said he. “it i"
“it is a noiseless lock,” said he. “it is the arrust which achand the crest in the rest roor and in the stried, and it have drack and the wrouse and it was one in here in the proarin. the hard the hard his light the drawiar and shall the there, and which i shild with 

# Testing the model

In [ ]:
def generate_text(length, diversity):
    # Get random starting text
    start_index = random.randint(0, len(text) - maxlen - 1)
    generated = ''
    sentence = text[start_index: start_index + maxlen]
    generated += sentence
    for i in range(length):
            x_pred = np.zeros((1, maxlen, len(chars)))
            for t, char in enumerate(sentence):
                x_pred[0, t, char_indices[char]] = 1.

            preds = model.predict(x_pred, verbose=0)[0]
            next_index = sample(preds, diversity)
            next_char = indices_char[next_index]

            generated += next_char
            sentence = sentence[1:] + next_char
    return generated

In [ ]:
print(generate_text(500, 0.2))

, since you refuse the most
excellent of the strange at the sident of the stainting the starting which was so surprised to the strange and strained and the dead to the strange and that i have a little strange and to the strange and to the strange and the strange and with the strange and starting over the strange and the connighting the stairs and the man who had she had been seen the dead and the straighter of the strange and the strong and strang and the companion that i have a show a little probable head and straind and strained and


# Model Evaluation

1. **Quantitative Evaluation**








In [ ]:
pip install torch transformers

In [ ]:
import torch
from transformers import GPT2LMHeadModel, GPT2Tokenizer

In [ ]:
# Load pre-trained model and tokenizer
model_name = "gpt2"
model = GPT2LMHeadModel.from_pretrained(model_name)
tokenizer = GPT2Tokenizer.from_pretrained(model_name)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [ ]:
model.eval()

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2SdpaAttention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=768, out_features=50257, bias=False)
)

In [ ]:
def calculate_perplexity(text):
    inputs = tokenizer.encode(text, return_tensors='pt')
    with torch.no_grad():
        outputs = model(inputs, labels=inputs)
        loss = outputs.loss
    perplexity = torch.exp(loss)
    return perplexity.item()

In [ ]:
example_text = "The quick brown fox jumps over the lazy dog."
perplexity_score = calculate_perplexity(example_text)
print(f"Perplexity: {perplexity_score}")

Perplexity: 162.47213745117188


2. **Qualitative Evaluation**

In [ ]:
def generate_sentence(prompt, max_length=50, num_return_sequences=5):
    inputs = tokenizer.encode(prompt, return_tensors='pt')
    outputs = model.generate(
        inputs,
        max_length=max_length,
        num_return_sequences=num_return_sequences,
        do_sample=True,
        top_k=50,
        top_p=0.95,
        temperature=0.7
    )
    generated_sentences = [tokenizer.decode(output, skip_special_tokens=True) for output in outputs]
    return generated_sentences

# Generate sentences for qualitative evaluation
prompt = "Once upon a time"
generated_sentences = generate_sentence(prompt)
for i, sentence in enumerate(generated_sentences):
    print(f"Generated Sentence {i + 1}: {sentence}")


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


Generated Sentence 1: Once upon a time, I was at a party when I saw a man with a gun in front of me. It was very odd. He was a young man, a very nice guy, and he was carrying a large gun. And I said
Generated Sentence 2: Once upon a time, the most important thing you could do was to start the discussion on your own. There are many people out there who will want to see a good discussion on this issue.

Now, there are many people out there who
Generated Sentence 3: Once upon a time, the world will have been made in your hands and you will have been created in your heart."

—Karen Foster, "The Miracle of the Birth of a Nation"

The Great Society of the North

Generated Sentence 4: Once upon a time, a man's body was found lying in a ditch on a road. A man came to his aid and was rescued. He had been shot in the back, and was lying on the ground with a bullet in his head.
Generated Sentence 5: Once upon a time, I was in the middle of a conversation with my husband about having a son. When h